We recommend that you use PyArrow to load your .parquet files. 

https://arrow.apache.org/docs/python/parquet.html

In [12]:
import pyarrow.parquet as pq
#Using PyArrow
df_train=pq.read_table("data/train.parquet")
df_valid=pq.read_table("data/valid.parquet")

df_train=df_train.to_pandas()
df_valid=df_valid.to_pandas()
#df_train.head()
#df_valid.head()

df_valid[target]=df_valid[target].replace("Wallet",int(0) )
df_valid[target]=df_valid[target].replace("Free Trial",int(1) )
df_valid[target]=df_valid[target].replace("Retail",int(2) )

ArrowIOError: Failed to open local file: data/valid.parquet , error: No such file or directory

Alternative Method: Make your own Spark Cluster.

In [5]:
import pyspark
sc = SparkContext.getOrCreate()
from pyspark.sql import *

import pandas as pd
import numpy as np

train=spark.read.parquet("data/train.parquet")
valid=spark.read.parquet("data/valid.parquet")
evaluation=spark.read.parquet("data/eval.parquet") #Students will not have access to this file.
evaluation_noTarget=spark.read.parquet("data/eval_noTarget.parquet")

print ("How many rows are in each file?")
print (train.count() )
print (valid.count() )
print (evaluation.count() )
print (evaluation_noTarget.count() )

#Turn into a Pandas DataFrame Iff you have Spark installed on your system
valid=valid.toPandas()

validCols=valid.columns
print (validCols)


target="subscription_sku_method"
valid[target]=valid[target].replace("Wallet",int(0) )
valid[target]=valid[target].replace("Free Trial",int(1) )
valid[target]=valid[target].replace("Retail",int(2) )

How many rows are in each file?
129550
1000
1000
1000
Index([u'sce_region_id', u'fulfillment_end_utc_dt_id',
       u'cur_subscription_renewal_type',
       u'subscription_sku_entitlement_period', u'subscription_sku_method',
       u'psn_tenure_month', u'total_playtime_seconds_91',
       u'total_playtime_seconds_31', u'total_playtime_seconds_14',
       u'total_playtime_seconds_7', u'total_igc_playtime_seconds_91',
       u'total_igc_playtime_seconds_31', u'total_igc_playtime_seconds_14',
       u'total_igc_playtime_seconds_7', u'playdays_91', u'playdays_31',
       u'playdays_14', u'playdays_7', u'total_online_seconds_91',
       u'total_online_seconds_31', u'total_online_seconds_14',
       u'total_online_seconds_7', u'online_playdays_91', u'stacked',
       u'ar_status', u'ad_status', u'cc_record_type', u'hist_lapsed',
       u'hist_retained', u'hist_reclaimed', u'store_spend_91',
       u'accumulated_igc_titles', u'purchased_discounted_titles',
       u'wallet_balance', u'cur_enti

Let's make a naive model, in which we predict the Users will always renew their 
subscription via their wallet funds!

In [17]:
naiveModel=np.zeros(len(valid[target]) ) #Always predict "Wallet"
df_naiveModel=np.zeros(len(df_valid[target]) )

#PyArrow Version
compareY=pd.DataFrame({"actual": df_valid[target] , "prediction":df_naiveModel})

#Spark Version of DataFrames
compareY=pd.DataFrame({"actual": valid[target] , "prediction":naiveModel})

print (compareY.head() )

   actual  prediction
0       0         0.0
1       0         0.0
2       0         0.0
3       0         0.0
4       1         0.0


Instead of using F1, or some other Classification Metric, we will be using 
our own cost function: How much does this misclassification cost Sony in 
either lost revenue, advertising, and coding times.


In [16]:
print ("Let's Build a Cost Function Specific to this use-case!")


def COST(row):
    """ Input the column names of your target variable 
    (actualCol) and the predicted value of the target (
    predCol) to compute this cost function."""
    actualCol="actual"
    predCol="prediction"
    if row[actualCol]==row[predCol]:
        return 0.0#If the prediction is correct, no problem.
    else:
        if row[predCol] == 0.0: #Prediction: User paid via Wallet
            if row[actualCol]==1.0: #Actual: User had a Free Trial
                return 12.0 # Strong negative reaction. Should have known!
            if row[actualCol]==2.0: #User went to the store and purchased
                return 0.25 #Little difference to Sony's bottom line. Minimal cost.
            
        if row[predCol] == 1.0: #Prediction: User has a Free Trial
            if row[actualCol]==0.0: #Actual: User paid via Wallet
                return 5.0 #User becomes annoyed at ads to renew their non-existent free trial
            if row[actualCol]==2.0: #Actual: User went to the store and purchased
                return 5.0 #User becomes annoyed at ads to renew their non-existent free trial
            
        if row[predCol] == 2.0: #Prediction: User purchases subscription via retail
            if row[actualCol]==0.0: #Actual: User paid via Wallet
                return 0.25 #Little difference to Sony's bottom line.
            if row[actualCol]==1.0: #Actual: User has a free trial
                return 12.0 # Large cost to Playstation, as they do not renew.


cost = compareY.apply( COST, axis=1).sum()
print ("Cost function: $%.2f" %cost)

Let's Build a Cost Function Specific to this use-case!
Cost function: $4297.25
